# ADNI_COMB 1.0 09/24/2019
##### Xiao Gao, Department of Radiology and Biomedical Imaging, UCSF; Myriam Chaumeil Lab (xiao.gao@ucsf.edu; xiao.gao@berkeley.edu)  


### Purpose: Reorganizing the latest open-sourced ADNI csv. files from http://adni.loni.usc.edu in Python environment; focusing on imaging data

### Prerequisite: Authorized access to ADNI database and having dowloaded necessary csv. files 

### Data Framework: Mainly based on adnimerge.csv by Michael C. Donohue, et al. from UCSD

### Notice: This document is presented by the author(s) as a service to ADNI data users. However, users should be aware that no formal review process has vetted this document and that ADNI cannot guarantee the accuracy or utility of this document.



## 0. Loading Libraries

In [36]:
import os 
import numpy as np 
import pandas as pd
from datetime import datetime
import time
import re

In [39]:
# A dumb function 1) locating the column of datatime in one pandas dateframe
#                 2) converting string datetime to Gregorian ordinal datetime
#                 3) adding the new numerical datetime as another column of one dataframe
#                 4) dropping rows without datetime information 
#                 5) returning the dataframe with alias prefix added

# Inputs: 1) df = one pandas dataframe read from an ADNI csv. file
#         2) alias = nickname of raw csv. file for convenience purposes in nameing and calling
#         3) date_entry_name = the specific column name of 'exam date' or 'scan date' of the raw csv. file
#         4) date_format = the format of input dateframe's datetime format. 
#            Usually, when generated via pd.read_csv method, the default datetime format is "%Y-%m-%d".

# Attention: input information about 'df', 'alias', and 'date_entry_name' is accessible from dataframe 'comb_df'

#_you_can_customize_the_numerical-datetime_column_name_here:
numdate_col = '_numdate'
#-----------------------------------------------------------


def add_numerical_date(df, date_entry_name, alias='new', date_format = "%Y-%m-%d", numdate_col=numdate_col):
    df.reset_index(drop=True, inplace=True)
    temp_date = df[date_entry_name]
    temp_numdate = np.zeros_like(temp_date)
    
    for i in range(len(temp_date)):
        date = temp_date[i]
        if type(date) == str:        
            if date[0:2]=='00':
                date='2'+date[1::] # handle dating problems like '0012-12-25' transferred from '12/12/25'    
            date = pd.to_datetime(date).date()
            date = str(date)
            df.loc[i, alias+numdate_col]= datetime.strptime(date, date_format).toordinal()
        else:
            df.drop(i, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
        
    return df
    

In [40]:
metrics_dict = ['SV', 'CV', 'SA', 'TA', 'TS', 'HS',\
                'MIN', 'MAX', 'AVG', 'SD', 'CT', 'MD',\
                'SUVR', 'VOLUME']
                # 'FA', 'MD', 'RD', 'AD',

def change_naming_convention(df, nm_pd, metrics_dict):
       
    for old_column in df.columns[100:4320]:
        new_column = old_column
        prefix = re.split('_', old_column)[0]
        number_convention = False
        string_convention = False
        whether_region_column  = False
        
        for nn in metrics_dict:
                # only brain region entries have specific metric names by the end of column name
                if bool(re.search('(?<![A-Z])'+nn+ '(?![A-Z])', old_column)):
                    whether_region_column = True
                    metric = nn
        
        if bool(re.search('ST[0-9]', old_column)): # tell numerical naming convention from string naming convention
            number_convention = True # some naming convention like 'ST39CV' or 'ST1SV'
        elif whether_region_column: 
            string_convention = True # some naming convention like 'LeftCaudalAnteriorCingulate_SUVR'
        
        
        if number_convention:
            
            try:
                # old_region = the string between '$prefix$_" and '$metric$'         
                old_region = re.search('(?<=' + prefix + '_)\w+(?=' + metric + ')', old_column).group(0)
                new_region = old_region
                for conv in nm_pd.columns:
                    if old_region in nm_pd[conv].values:
                        new_region = nm_pd.loc[nm_pd[conv]==old_region]['conv_comb'].values[0]
                new_column = prefix + '_' + new_region + '_' + metric
            except:
                print(old_column)
                print(metric)    
    
        if string_convention:
            
            try:
                # old_region = the text between '$prefix$_" and '_$metric$'
                old_region = re.search('(?<=' + prefix + '_)\w+(?=_' + metric + ')', old_column).group(0)
                new_region = old_region
                for conv in nm_pd.columns:
                    if old_region in nm_pd[conv].values:
                        new_region = nm_pd.loc[nm_pd[conv]==old_region]['conv_comb'].values[0]
                new_column = prefix + '_' + new_region + '_' + metric         
            except:
                print(old_column)
                print(metric)                     
                        
        df.rename(columns = {old_column:new_column}, inplace=True)
        
    return df

## 1. Checking downloaded csv. files

In [41]:
working_dir = os.path.dirname(os.getcwd()) # working from the upper directory of this .ipynd file
for root, dirs, files in os.walk(working_dir):
    for name in files:
        if name=="adnicomb_list.csv":
            comb_list = root + os.sep + name
        if name=="adnicomb_naming_convention.csv":
            nm_conv = root + os.sep + name
        
comb_df = pd.read_csv(comb_list, usecols=['csv', 'alias', 'date_entry', 'subject_entry', 'subject_type','recruit'])
nm_pd = pd.read_csv(nm_conv, usecols=['conv1', 'conv2', 'conv3', 'conv_comb'])

## 2. Locating csv. files locally

In [42]:
for root, dirs, files in os.walk(working_dir):
    for name in files:
        if name.endswith((".csv")):
            comb_df.loc[comb_df.csv == name, 'dir'] = root + os.sep + name

comb_df.head(20)            

,csv,alias,date_entry,subject_entry,subject_type,recruit,dir
0,ADNIMERGE.csv,merge,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/merge/...
1,MRILIST.csv,mrimeta,SCANDATE,SUBJECT,PTID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/mr/mr_...
2,PET_META_LIST.csv,petmeta,Scan Date,Subject,PTID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/pet/PE...
3,ADNI_HULAB.csv,csfhu,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/biomk/...
4,UPENNBIOMK9_04_19_17.csv,csfup9,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/biomk/...
5,UPENNBIOMK10_07_29_19.csv,csfupx,DRAWDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/biomk/...
6,DESIKANLAB.csv,desikan,STATIC,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/geneti...
7,UCSFFSL_02_01_16.csv,lv15one,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/mr/ucs...
8,UCSFFSL51ALL_08_01_16.csv,lv30two,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/mr/ucs...
9,UCSFFSX_11_02_15.csv,xv15one,EXAMDATE,RID,RID,1,/home/gavin/Documents/raj_lab/ADNI_COMB/mr/ucs...


## 3. Stacking dataframes over/alongside ADNIMERGE.csv

In [43]:
merge_alias = comb_df.loc[comb_df.csv == 'ADNIMERGE.csv', 'alias'].to_list()[0]

# import ADNIMERGE.csv as the framework/concordance of whole adnicomb dataset in format of pandas dateframe
adnicomb = (pd.read_csv(comb_df.loc[comb_df.alias==merge_alias,'dir'].to_list()[0])
              .add_prefix(merge_alias+'_')) # add prefix (the .csv's alias) to the columns

# add one column of Gregorian ordinal datetime
adnicomb = add_numerical_date(adnicomb, merge_alias+'_EXAMDATE', merge_alias,  '%Y-%m-%d')

# sort the dataframe in ascending order of RID as well as EXAMDATE
adnicomb.sort_values(by=[merge_alias+'_RID', merge_alias+'_EXAMDATE'], ascending=[1, 1], inplace = True)
adnicomb.reset_index(inplace = True, drop=True)
adnicomb_index =  adnicomb.index

# get lists of unique ADNI-1, Go, 2 and 3 subjects' RID and PTID
adni_rid_uniq = adnicomb[merge_alias+'_RID'].unique()
adni_ptid_uniq = adnicomb[merge_alias+'_PTID'].unique()

print('Cohort size of all ADNI phases: ', len(adni_rid_uniq))

/home/gavin/Library/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Cohort size of all ADNI phases:  2175


In [44]:
print('Reading csv. files:') 
for alias in comb_df.loc[comb_df.recruit==1]['alias']:
    start = time.time()
    flag_dupl=0
    print('... ',alias, ',')    
    if alias != merge_alias: # we have already import ADNIMERGE.csv
        csv_df =  (pd.read_csv(comb_df.loc[comb_df.alias==alias, 'dir'].to_list()[0])
                     .add_prefix(alias+'_'))
        
        date_entry = alias+'_'+comb_df.loc[comb_df.alias==alias, 'date_entry'].to_list()[0]
        subject_entry = alias+'_'+comb_df.loc[comb_df.alias==alias, 'subject_entry'].to_list()[0]
        subject_type = comb_df.loc[comb_df.alias==alias, 'subject_type'].to_list()[0]     
        
        # Here below, we use subject id and exam date as two different indices to locate clinical visit
        if subject_type == 'RID':
            index1 = merge_alias + '_RID'
            merge_subj_uniq = adni_rid_uniq

        elif subject_type == 'PTID':
            index1 = merge_alias + '_PTID'
            merge_subj_uniq = adni_ptid_uniq
            
        if 'VISCODE' in date_entry:
            index2 = merge_alias + '_VISCODE'
        elif 'STATIC' in date_entry:
            index2 = index1
            date_entry = subject_entry 
        else:
            index2 = merge_alias + numdate_col
            # if date type is numerical, add one column of Gregorian ordinal datetime to csv-of-interest
            csv_df = add_numerical_date(csv_df, date_entry, alias, "%Y-%m-%d")
            date_entry = alias + numdate_col 
        
        # initiate new columns from csv_df
        for new_col in csv_df.columns:
            #adnicomb[new_col]= np.nan  
            adnicomb[new_col]= np.full_like(np.empty((adnicomb.shape[0],1,)), np.nan).tolist()
        
        subj_date_merge = adnicomb[[index1,index2]]
        
        # unique csv. file subject IDs
        if any(csv_df[subject_entry].duplicated()):
            flag_dupl=1
        subject_uniq = csv_df[subject_entry].unique()
        
        # iterating through all patient_of_interests' IDs         
        for subject in subject_uniq:
            for new_col in csv_df.columns:
                 # For some screening studies, it is possible to find subjects of csv-of-interest abscent from ADNIMERGE.csv
                if subject in merge_subj_uniq:
                   # patient-of-interest's all clinical visit dates from ADNIMERGE.csv
                    try:
                        date_merge = subj_date_merge.loc[(subj_date_merge[index1]==subject).iloc[:,0]][index2].iloc[:,0]
                    except:
                        date_merge = subj_date_merge.loc[subj_date_merge[index1]==subject][index2]
                  
                    # patient-of-interest's all clinical visit information from csv-of-interest
                    col_csv = csv_df.loc[csv_df[subject_entry]==subject]
                    col_csv = col_csv.reset_index(drop=True)
                    col_csv.loc[col_csv[date_entry]=='sc',[date_entry]]='bl'  # consider screen visit as baseline visit
                  
                    date_csv = col_csv[date_entry].to_frame()
                    date_csv = (date_csv.reset_index(drop=True)
                                      .rename(columns={date_entry: "date"}))
                  
                    # initiate enrollment_datetime array, connecting date_csv to date_merge
                    date_enrol = pd.DataFrame(np.zeros_like(date_csv), columns = ['date'])
                  
                  
                  # an if-loop locating the scan/measure/draw date(s) best matching the record of ADNIMERGE.csv
                  # Attension: it's possible that several rows from csv-of-interest corrspond to one single row of
                  # ADNIMERGE.csv, so we need iterate through all dates from csv-of-interest to figure it out
                    for date1 in date_csv.date:
                      
                        if index2 == merge_alias + numdate_col:          
                            delta_date = np.absolute(date1 - date_merge)
                            enrol_index =  date_csv.date.loc[date_csv.date==date1].index.values
                            # dates from csv-of-interest are absorbed to the closest date from ADNIMERGE.csv
                            date_enrol.date.loc[enrol_index] = date_merge.iloc[np.argmin(delta_date.values)]               
                        else:
                          # if the datatime type is ordinal visit number, just find the same date string in ADNIMERGE.csv
                            if np.sum(date_merge == date1):  # if date1 in date_merge                         
                                date_merge_unique = date_merge.loc[date_merge==date1].iloc[0]
                                enrol_index =  date_csv.date.loc[date_csv.date==date1].index.values
                                date_enrol.date.loc[enrol_index] = date_merge_unique
                            else:
                                # drop the csv date not existing in merge date, which rarely happens except for screening visits
                                drop_index = date_csv.date.loc[date_csv.date==date1].index.values
                                date_enrol.drop(drop_index, inplace=True)                                            
  
                    for date2 in date_enrol.date.unique():                                                     
                       # locate the row to be edited in adnicomb based on subject's ID and visit's date
                        try:
                            edit_boolean = np.logical_and(subj_date_merge[index1].iloc[:,0]==subject, subj_date_merge[index2].iloc[:,0]==date2)
                            edit_row = adnicomb_index[edit_boolean].values
                        except:
                            edit_boolean = np.logical_and(subj_date_merge[index1]==subject, subj_date_merge[index2]==date2)
                            edit_row = adnicomb_index[edit_boolean].values
                            
                        # selected row in csv. file of interest based on date_enrol
                        enrol_row = date_enrol.date.loc[date_enrol.date==date2].index.values
                        enrol_list = col_csv.loc[enrol_row][new_col].to_list()
                        
                        # If one single clinical follow-up corresponds to multiple records from the csv-of-interest,
                        # all those records are warpped up into a list and then inserted to that follow-up's dataframe cell   
                        if flag_dupl==1: 
                            #adnicomb.loc[edit_row, new_col] = enrol_list  
                            #--> not working when trying to insert a 'list' into a dataframe cell
                            
                            adnicomb.loc[edit_row,new_col] = adnicomb.loc[edit_row, new_col].apply(lambda x: enrol_list)
                        else:
                            adnicomb.loc[edit_row,new_col] = adnicomb.loc[edit_row, new_col].apply(lambda x: enrol_list)
                            
                            #adnicomb.loc[edit_row, new_col] = enrol_list 
                            #--> it works, but we'd better keep all dataframe cells as type of 'list',
                            # for convinience purposes in terms of future data calling
                
    end = time.time()        
    print('    with using ', '{:.{prec}f}'.format(end -start, prec=2), ' seconds ...')        
        

Reading csv. files:
...  merge ,
    with using  0.00  seconds ...
...  mrimeta ,


/home/gavin/Library/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


    with using  1332.53  seconds ...
...  petmeta ,
    with using  708.45  seconds ...
...  csfhu ,
    with using  76.02  seconds ...
...  csfup9 ,
    with using  246.99  seconds ...
...  csfupx ,
    with using  49.10  seconds ...
...  desikan ,
    with using  84.66  seconds ...
...  lv15one ,
    with using  7132.26  seconds ...
...  lv30two ,
    with using  3959.64  seconds ...
...  xv15one ,
    with using  8118.86  seconds ...
...  xv30one ,
    with using  1095.87  seconds ...
...  xv30two ,
    with using  8354.22  seconds ...
...  xv3three ,
    with using  1866.51  seconds ...
...  dmn ,
    with using  405.42  seconds ...
...  infarct ,
    with using  529.27  seconds ...
...  dti ,
    with using  1121.73  seconds ...
...  asl ,
    with using  2903.65  seconds ...
...  tbm ,
    with using  562.56  seconds ...
...  taunpvc ,
    with using  1722.24  seconds ...
...  tauwpvc ,
    with using  1179.91  seconds ...
...  av45 ,
    with using  4434.88  seconds ...
...  fbb

/home/gavin/Library/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,11,16,32,33,38,52,66,67,72,84,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


    with using  7645.55  seconds ...
...  p180fi1 ,
    with using  1365.90  seconds ...
...  p180fi2 ,
    with using  1681.35  seconds ...
...  p180lc1 ,
    with using  471.61  seconds ...
...  p180lc2 ,
    with using  671.82  seconds ...
...  lipopro ,
    with using  4552.89  seconds ...
...  etn ,
    with using  809.50  seconds ...
...  apoe ,
    with using  381.05  seconds ...
...  ab42_40 ,
    with using  203.81  seconds ...
...  nfl1 ,
    with using  67.24  seconds ...
...  nfl2 ,
    with using  287.62  seconds ...
...  csfzhang ,
    with using  89.42  seconds ...
...  s_trem ,
    with using  238.78  seconds ...
...  csfeuro ,
    with using  20.32  seconds ...
...  csffagan ,
    with using  67.89  seconds ...
...  msms1 ,
    with using  41.99  seconds ...
...  msms2 ,
    with using  127.07  seconds ...


## 4. Unifying naming convention of brain regions 

In [45]:
adnicomb = change_naming_convention(adnicomb, nm_pd, metrics_dict)

dti_MD_CST_L
MD
dti_MD_CST_R
MD
dti_MD_ICP_L
MD
dti_MD_ICP_R
MD
dti_MD_ML_L
MD
dti_MD_ML_R
MD
dti_MD_SCP_L
MD
dti_MD_SCP_R
MD
dti_MD_CP_L
MD
dti_MD_CP_R
MD
dti_MD_ALIC_L
MD
dti_MD_ALIC_R
MD
dti_MD_PLIC_L
MD
dti_MD_PLIC_R
MD
dti_MD_PTR_L
MD
dti_MD_PTR_R
MD
dti_MD_ACR_L
MD
dti_MD_ACR_R
MD
dti_MD_SCR_L
MD
dti_MD_SCR_R
MD
dti_MD_PCR_L
MD
dti_MD_PCR_R
MD
dti_MD_CGC_L
MD
dti_MD_CGC_R
MD
dti_MD_CGH_L
MD
dti_MD_CGH_R
MD
dti_MD_FX_ST_L
MD
dti_MD_FX_ST_R
MD
dti_MD_SLF_L
MD
dti_MD_SLF_R
MD
dti_MD_SFO_L
MD
dti_MD_SFO_R
MD
dti_MD_IFO_L
MD
dti_MD_IFO_R
MD
dti_MD_SS_L
MD
dti_MD_SS_R
MD
dti_MD_EC_L
MD
dti_MD_EC_R
MD
dti_MD_UNC_L
MD
dti_MD_UNC_R
MD
dti_MD_FX_L
MD
dti_MD_FX_R
MD
dti_MD_GCC_L
MD
dti_MD_GCC_R
MD
dti_MD_BCC_L
MD
dti_MD_BCC_R
MD
dti_MD_SCC_L
MD
dti_MD_SCC_R
MD
dti_MD_RLIC_L
MD
dti_MD_RLIC_R
MD
dti_MD_TAP_L
MD
dti_MD_TAP_R
MD
dti_MD_SUMGCC
MD
dti_MD_SUMBCC
MD
dti_MD_SUMSCC
MD
dti_MD_SUMCC
MD
dti_MD_SUMFX
MD


In [46]:
# chang CerebellumCortex_SV etc. to CerebellumCortex_CV etc.
for prefix in ['xv15one','xv30one','xv30two','xv3three']:
    for region in ['CerebellumCortex', 'Thalamus','Caudate','Putamen',
                   'Pallidum','Hippocampus','Amygdala','AccumbensArea', 'VentralDC']:
                adnicomb.rename(
                        columns={prefix+'_Right'+region+'_SV': prefix+'_Right'+region+'_CV'}, 
                        inplace=True)
                adnicomb.rename(
                        columns={prefix+'_Left'+region+'_SV': prefix+'_Left'+region+'_CV'}, 
                        inplace=True)

## 5. Saving ADNICOMB as pickle 

In [47]:
# saving generated dataframe to pickle
adnicomb.to_pickle(working_dir+'/adnicomb_v1_5.pkl')

In [34]:
adnicomb = pd.read_pickle(working_dir+'/adnicomb_v1_5.pkl')

In [11]:
# Double check whether one certain csv. files has been imported accurately
# (only non-NaN elements shown here)
i=18
temp_alias = comb_df.alias[i]
temp_entry = comb_df.subject_entry[i]
print(temp_alias, temp_entry)
temp_pd = (adnicomb[['merge_RID','merge_EXAMDATE', 'merge_DX']]
                   .join(adnicomb.filter(regex='^'+temp_alias, axis=1)))
temp_pd = temp_pd.loc[temp_pd[temp_alias + '_' + temp_entry].notnull()]
temp_pd.head()

taunpvc RID


,merge_RID,merge_EXAMDATE,merge_DX,taunpvc_RID,taunpvc_VISCODE,taunpvc_VISCODE2,taunpvc_EXAMDATE,taunpvc_InferiorCerebellum_SUVR,taunpvc_InferiorCerebellum_VOLUME,taunpvc_ErodedSubcorticalWM_SUVR,...,taunpvc_RightThalamus_SUVR,taunpvc_RightThalamus_VOLUME,taunpvc_RightVentralDC_SUVR,taunpvc_RightVentralDC_VOLUME,taunpvc_RightVessel_SUVR,taunpvc_RightVessel_VOLUME,taunpvc_WMHypoIntensities_SUVR,taunpvc_WMHypoIntensities_VOLUME,taunpvc_update_stamp,taunpvc_numdate
109,21,2017-11-27,CN,[21],[init],[m144],[2018-02-02],[1.0299],[52274],[1.2078],...,[1.4502],[5968],[1.37],[3045],[1.4225],[55.0],[1.0307],[3229],[2019-08-28 15:59:33.0],[736727]
162,31,2018-04-17,CN,[31],[init],[m144],[2018-04-24],[0.9597],[47466],[1.0607],...,[1.1399],[5442],[1.2306],[3083],[1.3924],[25.0],[0.7766],[31427],[2019-08-28 15:59:33.0],[736808]
163,31,2019-04-23,CN,[31],[y1],[m156],[2019-04-23],[0.9620000000000001],[47466],[1.0461],...,[1.0777],[5442],[1.1674],[3083],[1.4014],[25.0],[0.7909],[31427],[2019-08-28 15:59:33.0],[737172]
323,56,2017-11-28,MCI,[56],[init],[m144],[2018-02-20],[0.9873],[56231],[1.2592],...,[1.3359],[6143],[1.3548],[3243],[1.5292],[98.0],[1.035],[2227],[2019-08-28 15:59:33.0],[736745]
324,56,2019-01-10,MCI,[56],[y1],[m156],[2019-01-10],[0.9621],[56231],[1.2903],...,[1.364],[6143],[1.3791],[3243],[1.4749],[98.0],[1.0708],[2227],[2019-08-28 15:59:33.0],[737069]
